In [1]:
import pandas as pd
import numpy as np

In [2]:
import string

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Load Amazon Dataset

Load the dataset consisting of baby product reviews on Amazon.com. Store the data in a data frame **products**.

In [4]:
products = pd.read_csv('./data/amazon_baby.csv')

In [5]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [6]:
products.shape

(183531, 3)

# Perform Text Cleaning

We start by removing punctuation, so that words "cake." and "cake!" are counted as the same word.

- Write a function **remove_punctuation** that strips punctuation from a line of text
- Apply this function to every element in the **review** column of **products**, and save the result to a new column **review_clean**.

**Aside.** In this notebook, we remove all punctuation for the sake of simplicity. A smarter approach to punctuation would preserve phrases such as "I'd", "would've", "hadn't" and so forth.

In [7]:
def remove_punctuation(text):
    tran_tab = str.maketrans('', '', string.punctuation)
    return text.translate(tran_tab) 

**MPORTANT.** Make sure to fill n/a values in the **review** column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the **review** columns.

In [8]:
products.fillna({'review':''}, inplace=True)

In [9]:
products['review_clean'] = products.review.apply(remove_punctuation)

# Extract Sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [10]:
idx = (products.rating != 3)
products = products[idx]

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the **rating** column.

In [11]:
products['sentiment'] = products.rating.apply(
    lambda rating: +1 if rating > 3 else -1
)

Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

In [12]:
products.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


# Train/Test Split

In [13]:
idx_train = pd.read_json('./data/module-2-assignment-train-idx.json')[0]
train_data = products.iloc[idx_train,:]

In [14]:
idx_test = pd.read_json('./data/module-2-assignment-test-idx.json')[0]
test_data = products.iloc[idx_test,:]

# Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

- Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
- Compute the occurrences of the words in each review and collect them into a row vector.
- Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.
- Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

The following cell uses CountVectorizer in scikit-learn. Notice the **token_pattern** argument in the constructor.

In [15]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

Keep in mind that the test data must be transformed in the same way as the training data.

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data.

Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the [LogisticRegression class](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and then call the method fit() to train the classifier. This model should use the sparse word count matrix (**train_matrix**) as features and the column **sentiment** of **train_data** as the target. Use the default values for other parameters. Call this model **sentiment_model**.

In [16]:
sentiment_model = (
    LogisticRegression(multi_class='ovr', solver='liblinear').
    fit(train_matrix, train_data.sentiment)
)

S:\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


There should be over 100,000 coefficients in this **sentiment_model**. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

In [17]:
np.sum(sentiment_model.coef_ >= 0)

85877

# Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 data points in the test data. Take the 11th, 12th, and 13th data points in the test data and save them to **sample_test_data**.

In [18]:
sample_test_data = test_data.iloc[10:13,:]
sample_test_data

,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [19]:
sample_test_data.iloc[0,:]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [20]:
sample_test_data.iloc[1,:]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a class prediction for the **sample_test_data**. The **sentiment_model** should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the score (sometimes called margin) for the logistic regression model is defined as:
$$
\text{score}_{i} = w^T h(x_i)
$$
where $h(x_i)$ represents the features for data point i. We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in **sample_test_data**. In scikit-learn, you can call the decision_function() function.

In [21]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.60840687  -3.12665506 -10.42354879]


# Prediciting Sentiment

Using scores, write code to calculate predicted labels for **sample_test_data**.

In [22]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [23]:
sample_test_data.sentiment

59    1
71   -1
91   -1
Name: sentiment, dtype: int64

# Probability Predictions

Using the scores calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range [0, 1].

In [24]:
sentiment_model.predict_proba(sample_test_matrix)[:,1]

array([9.96346491e-01, 4.20210525e-02, 2.97233236e-05])

# Find the most positive (and negative) review

Using the sentiment_model, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

In [25]:
pr_hat = sentiment_model.predict_proba(test_matrix)[:,1]

In [26]:
idx_top20 = np.argsort(1-pr_hat)[:20]

In [27]:
test_data.iloc[idx_top20,:].name

180646        Mamas &amp; Papas 2014 Urbo2 Stroller - Black
52631     Evenflo X Sport Plus Convenience Stroller - Ch...
147949    Baby Jogger City Mini GT Single Stroller, Shad...
137034           Graco Pack 'n Play Element Playard - Flint
80155     Simple Wishes Hands-Free Breastpump Bra, Pink,...
87017       Baby Einstein Around The World Discovery Center
133651                    Britax 2012 B-Agile Stroller, Red
168081    Buttons Cloth Diaper Cover - One Size - 8 Colo...
97325     Freemie Hands-Free Concealable Breast Pump Col...
100166    Infantino Wrap and Tie Baby Carrier, Black Blu...
50315            P'Kolino Silly Soft Seating in Tias, Green
140816           Diono RadianRXT Convertible Car Seat, Plum
168697    Graco FastAction Fold Jogger Click Connect Str...
114796    Fisher-Price Cradle 'N Swing,  My Little Snuga...
119182    Roan Rocco Classic Pram Stroller 2-in-1 with B...
66059          Evenflo 6 Pack Classic Glass Bottle, 4-Ounce
22586        Britax Decathlon Convertibl

Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the 20 reviews in the test_data with the lowest probability of being classified as a positive review. Repeat the same steps above but make sure you sort in the opposite order.

In [28]:
idx_bottom20 = np.argsort(pr_hat)[:20]

In [29]:
test_data.iloc[idx_bottom20,:].name

16042           Fisher-Price Ocean Wonders Aquarium Bouncer
120209    Levana Safe N'See Digital Video Baby Monitor w...
77072        Safety 1st Exchangeable Tip 3 in 1 Thermometer
48694     Adiri BPA Free Natural Nurser Ultimate Bottle ...
155287    VTech Communications Safe &amp; Sounds Full Co...
94560     The First Years True Choice P400 Premium Digit...
53207                   Safety 1st High-Def Digital Monitor
81332                 Cloth Diaper Sprayer--styles may vary
113995    Motorola Digital Video Baby Monitor with Room ...
10677                     Philips AVENT Newborn Starter Set
59546                Ellaroo Mei Tai Baby Carrier - Hershey
9915           Cosco Alpha Omega Elite Convertible Car Seat
172090    Belkin WeMo Wi-Fi Baby Monitor for Apple iPhon...
75994            Peg-Perego Tatamia High Chair, White Latte
40079     Chicco Cortina KeyFit 30 Travel System in Adve...
149987                     NUK Cook-n-Blend Baby Food Maker
154878    VTech Communications Safe &amp

# Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier.

This can be computed as follows:

- Step 1: Use the sentiment_model to compute class predictions.
- Step 2: Count the number of data points when the predicted class labels match the ground truth labels.
- Step 3: Divide the total number of correct predictions by the total number of data points in the dataset.

In [30]:
y_hat = sentiment_model.predict(test_matrix)

In [31]:
np.mean(y_hat == test_data.sentiment)

0.9321154307655387

# Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected 20 words to work with. These are:

In [32]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [33]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words

Compute word count vectors for the training and test data and obtain the sparse matrices train_matrix_word_subset and test_matrix_word_subset, respectively.

In [34]:
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

# Train a logistic regression model on a subset of data

Now build a logistic regression classifier with train_matrix_word_subset as features and sentiment as the target. Call this model simple_model.

In [35]:
simple_model = (
    LogisticRegression(multi_class='ovr', solver='liblinear').
    fit(train_matrix_word_subset, train_data.sentiment)
)

Let us inspect the weights (coefficients) of the simple_model. First, build a table to store (word, coefficient) pairs.

In [36]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                        'coefficient':simple_model.coef_.flatten()})
simple_model_coef_table.sort_values(by='coefficient', ascending=False)

,word,coefficient
6,loves,1.673074
5,perfect,1.509812
0,love,1.363690
2,easy,1.192538
1,great,0.944000
4,little,0.520186
7,well,0.503760
8,able,0.190909
3,old,0.085513
9,car,0.058855


In [37]:
np.sum(simple_model.coef_ >= 0)

10

In [38]:
pos_words = simple_model_coef_table.word[simple_model_coef_table.coefficient >= 0]

In [39]:
all_words = vectorizer.get_feature_names()

In [40]:
idx = [all_words.index(w) for w in pos_words]

In [41]:
whole_model_coef_table = pd.DataFrame(
    {
        'word' : pos_words,
        'coefficient' : sentiment_model.coef_.flatten()[idx]
    }
)
whole_model_coef_table

,word,coefficient
0,love,1.578642
1,great,1.228073
2,easy,1.356873
3,old,0.053091
4,little,0.638586
5,perfect,1.862930
6,loves,1.517420
7,well,0.542647
8,able,0.389431
9,car,0.126364


In [42]:
all(whole_model_coef_table.coefficient >= 0)

True

# Comparing models

We will now compare the accuracy of the sentiment_model and the simple_model.

First, compute the classification accuracy of the sentiment_model on the train_data.

In [43]:
np.mean(sentiment_model.predict(train_matrix) == train_data.sentiment)

0.967934880374168

Now, compute the classification accuracy of the simple_model on the train_data.

In [44]:
np.mean(simple_model.predict(train_matrix_word_subset) == train_data.sentiment)

0.8668225700065959

Now, we will repeat this exercise on the test_data. Start by computing the classification accuracy of the sentiment_model on the test_data.

In [45]:
np.mean(sentiment_model.predict(test_matrix) == test_data.sentiment)

0.9321154307655387

Next, compute the classification accuracy of the simple_model on the test_data.

In [46]:
np.mean(simple_model.predict(test_matrix_word_subset) == test_data.sentiment)

0.8693604511639069

# Baseline: Majority class prediction

It is quite common to use the majority class classifier as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

In [47]:
train_data.sentiment.value_counts()

 1    112164
-1     21252
Name: sentiment, dtype: int64

In [48]:
np.mean(test_data.sentiment == 1)

0.8427825773938085